In [79]:
achievements = [
    {"title": "wandering shroom", "stam_cost": 15, "wing_cost": 2, "remaining": 0},
    {"title": "catoblepas anemos", "stam_cost": 20, "wing_cost": 2, "remaining": 19},
    {"title": "eolian phantom", "stam_cost": 25, "wing_cost": 2, "remaining": 0},
    {"title": "proud manticore", "stam_cost": 25, "wing_cost": 2, "remaining": 3},
    {"title": "ebon chimera", "stam_cost": 30, "wing_cost": 3, "remaining": 1}
]

In [80]:
from datetime import datetime

time_remaining = datetime(2020, 2, 27) - datetime.now()
hours_remaining, remainder = divmod(time_remaining.seconds, 3600)
hours_remaining += time_remaining.days * 24
minutes, seconds = divmod(remainder, 60)
print(hours_remaining, minutes, seconds)

56 56 34


In [81]:
current_stam = 120
current_wings = 5
max_stam = 120

# Real regen: 10/h; Honey Tea: 2/h
total_remaining_stam = current_stam + 12 * hours_remaining + minutes // 6 + minutes // 30
total_remaining_stam

802

In [82]:
cost_to_finish = sum([i['remaining'] * i['stam_cost'] for i in achievements])
cost_to_finish

485

In [83]:
total_difference = cost_to_finish - total_remaining_stam
total_difference

-317

In [84]:
[i["title"] for i in achievements]

['wandering shroom',
 'catoblepas anemos',
 'eolian phantom',
 'proud manticore',
 'ebon chimera']

In [85]:
f = [i["title"] for i in achievements]
double_drops = [
    #[0, 1, 2, 3, 4],
    # [0, ],
    # [1,3,],
    # [0,2],
    [4],
    [1,3],
    [2,4],
    [0,1,2,3,4]
]

double_drops = [[f[j] for j in i] for i in double_drops]

from datetime import date, timedelta, time
from dateutil.rrule import rrule, DAILY

a = datetime.now()
b = date(2020, 2, 27)

double_drops_dict = {}

for index, dt in enumerate(rrule(DAILY, dtstart=a, until=b)):
    double_drops_dict[dt.strftime("%Y-%m-%d")] = {
        "double_drops": double_drops[index],
        "remaining_stam": 12*24 - 15*3 - 20*2 - 25*2,
        "remaining_wings": 24
    }

hours_left_today = datetime.combine(date.today() + timedelta(days=1), time.min) - datetime.now()
hours_left_today = hours_left_today.seconds // 3600
double_drops_dict[datetime.now().strftime("%Y-%m-%d")]["remaining_stam"] = current_stam + 10*hours_left_today
double_drops_dict[datetime.now().strftime("%Y-%m-%d")]["remaining_wings"] = current_wings + hours_left_today 
double_drops_dict

{'2020-02-24': {'double_drops': ['ebon chimera'],
  'remaining_stam': 200,
  'remaining_wings': 13},
 '2020-02-25': {'double_drops': ['catoblepas anemos', 'proud manticore'],
  'remaining_stam': 153,
  'remaining_wings': 24},
 '2020-02-26': {'double_drops': ['eolian phantom', 'ebon chimera'],
  'remaining_stam': 153,
  'remaining_wings': 24}}

In [88]:
import copy
from IPython.display import display, HTML
from pandas import DataFrame

track_achieves = copy.deepcopy(achievements)
remaining_stamina = 0
remaining_wings = 0
for i in double_drops_dict.keys():
    display(HTML(f"<h2>Prospective Schedule for {i}</h2><details><summary>Click to expand</summary>Some text</details>"))
    todays_stam = double_drops_dict[i]["remaining_stam"]
    todays_wings = double_drops_dict[i]["remaining_wings"]
    outputs = []
    display(DataFrame([
        {"Type": "Stamina", "Regened": todays_stam, "Remaining": remaining_stamina},
        {"Type": "Wings", "Regened": todays_wings, "Remaining": remaining_wings}]))
    todays_stam += remaining_stamina
    todays_wings += remaining_wings
    todays_double_drops = double_drops_dict[i]["double_drops"]
    prospective_fights = [i for i in track_achieves if i["remaining"] > 0 and i["title"] in todays_double_drops]
    display(HTML("<h4>Prospective Fights</h4>"))
    display(DataFrame(prospective_fights))
    display(HTML("<h4>Suggested Schedule</h4>"))
    stam_costs = [i["stam_cost"] for i in prospective_fights]
    stam_costs.sort(reverse=True)
    fights_to_run = []
    while stam_costs and any([True for i in track_achieves if i["title"] in todays_double_drops and i["remaining"] > 0]) and todays_stam > min(stam_costs):
        for stam in stam_costs:
            wing_cost = 3 if stam == 30 else 2
            if stam > todays_stam:
                if wing_cost > todays_wings:
                    continue
            for fight in track_achieves:
                if fight["stam_cost"] != stam or fight["remaining"] <= 0 or not fight["title"] in todays_double_drops:
                    # if fight["stam_cost"] == stam and fight['remaining'] > 0 and not fight["title"] in todays_double_drops:
                    #    outputs.append(HTML(f"<p>Skipping {fight['title']}</p>"))
                    continue
                runs = min(todays_stam // stam, fight["remaining"])
                fight["remaining"] -= runs
                coops = min(todays_wings // wing_cost, fight["remaining"])
                fights_to_run.append(
                    {"Title": fight["title"], "Solo Runs": runs, "Coops": coops, "Stamina": stam*runs, "Wings": coops*wing_cost})
                fight["remaining"] -= coops
                break
            todays_stam -= runs*stam
            todays_wings -= coops*wing_cost
    remaining_stamina = todays_stam if stam_costs else 0
    remaining_wings = min(todays_wings, 2)
    if todays_wings - remaining_wings > 0:
        fights_to_run.append(
            {"Title": "Dragon Trials", "Solo Runs": "", "Coops": "", "Stamina": "", "Wings": todays_wings - remaining_wings})
    if not stam_costs:
        fights_to_run.append({"Title": "Solo Stuff", "Solo Runs": "", "Coops": "", "Stamina": todays_stam, "Wings": ""})
    outputs.append(DataFrame(fights_to_run))
    outputs.append(DataFrame([{"Remaining": "Stamina", "Qty": remaining_stamina}, {"Remaining": "Wings", "Qty": remaining_wings}]))
    for i in outputs:
        display(i)
    display(HTML("</details>"))

display(HTML("<hr/>"))
display(DataFrame([i for i in track_achieves if i['remaining'] > 0]))
print("Remaining fights:", [i["title"] for i in track_achieves if i["remaining"] > 0])
missing_stam = sum([i["stam_cost"] * i["remaining"] for i in track_achieves])
print("Required Stamina:", missing_stam, "=", missing_stam / 30, "big honeys.")

,Type,Regened,Remaining
0,Stamina,200,0
1,Wings,13,0


,title,stam_cost,wing_cost,remaining
0,ebon chimera,30,3,1


,Title,Solo Runs,Coops,Stamina,Wings
0,ebon chimera,1,0,30,0
1,Dragon Trials,,,,11


,Remaining,Qty
0,Stamina,170
1,Wings,2


,Type,Regened,Remaining
0,Stamina,153,170
1,Wings,24,2


...

""


,Title,Solo Runs,Coops,Stamina,Wings
0,Dragon Trials,,,,24
1,Solo Stuff,,,161,


,Remaining,Qty
0,Stamina,0
1,Wings,2


""


Remaining fights: []
Required Stamina: 0 = 0.0 big honeys.
